In [1]:
# Znak wykrzyknika (!) pozwala wywołać komendy systemowe Linuxa
!git clone https://github.com/WebCrawlerr/GAT.git


Cloning into 'GAT'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 83 (delta 41), reused 61 (delta 26), pack-reused 0 (from 0)
Receiving objects: 100% (83/83), 54.35 KiB | 1.43 MiB/s, done.
Resolving deltas: 100% (41/41), done.


In [2]:
# Pobranie zmian
!git pull origin main


fatal: not a git repository (or any parent up to mount point /kaggle)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).


In [3]:
# Wyświetla listę plików w pobranym folderze
!ls -R  GAT

GAT:
main.py  requirements.txt  src	tests

GAT/src:
config.py	    dataset.py	 model.py     train.py
data_processing.py  features.py  __pycache__  utils.py

GAT/src/__pycache__:
config.cpython-312.pyc		 features.cpython-312.pyc
data_processing.cpython-312.pyc  train.cpython-312.pyc
dataset.cpython-312.pyc		 utils.cpython-312.pyc

GAT/tests:
test_cv.py  test_features.py  test_pipeline.py	test_plotting.py


In [4]:
import shutil
import os
import sys
# 1. Zmiana katalogu roboczego (odpowiednik komendy 'cd')
os.chdir('./GAT')
print(f"Aktualny katalog: {os.getcwd()}")

Aktualny katalog: /kaggle/working/GAT


In [5]:
# Najpierw sprawdźmy wersję po restarcie (czasem GPU ma starszego PyTorcha)
import torch
version = torch.__version__
print(f"PyTorch version: {version}")

# Instalacja PyG i bibliotek pomocniczych (korzystamy z gotowych paczek .whl, żeby nie kompilować przez 20 min)
# UWAGA: Ta komenda dynamicznie dobiera link do wersji Twojego PyTorcha
torch_version_suffix = version.split('+')[0].replace('.', '') # np. 260 lub 240
cuda_version_suffix = 'cu' + torch.version.cuda.replace('.', '') # np. cu121

print(f"Instaluję dla: torch-{torch_version_suffix} + {cuda_version_suffix}")

# Instalacja zależności
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv \
  -f https://data.pyg.org/whl/torch-{version}.html

# Instalacja reszty z Twojego pliku (bez ponownego instalowania torch-geometric, bo zrobiliśmy to wyżej)
!pip install -r requirements.txt

PyTorch version: 2.6.0+cu124
Instaluję dla: torch-260 + cu124
Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 75.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 115.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:

In [6]:
import torch
import torch_geometric

print(f"CUDA dostępne: {torch.cuda.is_available()}")
print(f"Wersja PyTorch: {torch.__version__}")
print(f"Wersja PyG: {torch_geometric.__version__}")

# Próba importu kluczowych warstw
try:
    from torch_geometric.nn import GATConv
    print("✅ GATConv zaimportowany pomyślnie!")
except ImportError as e:
    print(f"❌ Błąd importu GATConv: {e}")

# Sprawdzenie czy mamy backendy (scatter/sparse)
# PyG 2.7+ potrafi czasem działać bez nich (wolniej), ale warto sprawdzić
try:
    import torch_scatter
    print("✅ torch_scatter obecny")
except ImportError:
    print("⚠️ torch_scatter BRAK (Może to spowolnić trening lub wywołać błąd)")

try:
    import torch_sparse
    print("✅ torch_sparse obecny")
except ImportError:
    print("⚠️ torch_sparse BRAK (Wymagany dla niektórych operatorów)")

CUDA dostępne: True
Wersja PyTorch: 2.6.0+cu124
Wersja PyG: 2.7.0
✅ GATConv zaimportowany pomyślnie!
✅ torch_scatter obecny
✅ torch_sparse obecny


In [7]:
!python main.py --raw_file /kaggle/input/bindingdb-smiles/BindingDB_All.tsv --processed_dir ./data/processed --cv 5

Starting GAT BRD4 Binding Prediction Pipeline...
Loading and filtering data from /kaggle/input/bindingdb-smiles/BindingDB_All.tsv in chunks of 100000...
Processed 1000000 rows...
Processed 2000000 rows...
Processed 3000000 rows...
Finished processing 3078912 rows.
Found 22758 records for BRD4.
Cleaning and Labeling...
Final dataset size: 22616
Creating Graph Dataset in ./data/processed (this may take a while)...
Processing...
Processing Molecules:  12%|█▋             | 2618/22616 [00:03<00:26, 750.90it/s][00:45:53] Can't kekulize mol.  Unkekulized atoms: 1 4 6
[00:45:53] Can't kekulize mol.  Unkekulized atoms: 18 19 23
[00:45:53] Can't kekulize mol.  Unkekulized atoms: 23 24 28
[00:45:53] Can't kekulize mol.  Unkekulized atoms: 1 4 6
[00:45:53] Can't kekulize mol.  Unkekulized atoms: 20 21 25
[00:45:53] Can't kekulize mol.  Unkekulized atoms: 20 21 25
[00:45:53] Can't kekulize mol.  Unkekulized atoms: 1 4 6
[00:45:53] Can't kekulize mol.  Unkekulized atoms: 1 4 6
[00:45:53] Can't kekul

In [8]:
import os
import shutil
from IPython.display import FileLink

# Nazwa pliku, którego szukamy
zip_name = 'GAT_BRD4_Results.zip'

# 1. Znajdź plik w całym systemie plików roboczych
found_path = None
for root, dirs, files in os.walk('/kaggle/working'):
    if zip_name in files:
        found_path = os.path.join(root, zip_name)
        break

if found_path:
    print(f"✅ Znaleziono plik tutaj: {found_path}")
    
    # 2. Przenieś go do głównego katalogu (Dla pewności)
    target_path = f'/kaggle/working/{zip_name}'
    
    if found_path != target_path:
        shutil.move(found_path, target_path)
        print(f"📦 Przeniesiono plik do głównego katalogu: {target_path}")
    
    # 3. Zmień katalog roboczy na główny (żeby FileLink zadziałał)
    os.chdir('/kaggle/working')
    
    # 4. Wyświetl link
    print("👇 Kliknij poniżej:")
    display(FileLink(zip_name))
else:
    print("❌ Nie znaleziono pliku ZIP. Upewnij się, że poprzednia komórka wykonała się bez błędów.")
    # Sprawdźmy co w ogóle mamy na dysku
    print("Lista plików w obecnym folderze:")
    print(os.listdir('.'))

❌ Nie znaleziono pliku ZIP. Upewnij się, że poprzednia komórka wykonała się bez błędów.
Lista plików w obecnym folderze:
['fold_4_roc_curve.png', 'fold_3_training_curves.png', 'requirements.txt', '.gitignore', 'models', 'fold_2_pr_curve.png', 'fold_0_pr_curve.png', 'fold_1_pr_curve.png', 'fold_4_pr_curve.png', 'data', 'fold_0_confusion_matrix.png', 'fold_0_roc_curve.png', 'fold_0_training_curves.png', 'fold_2_training_curves.png', 'fold_1_training_curves.png', 'fold_3_roc_curve.png', 'fold_1_roc_curve.png', 'fold_3_pr_curve.png', 'fold_2_confusion_matrix.png', 'fold_4_training_curves.png', 'fold_4_confusion_matrix.png', 'fold_3_confusion_matrix.png', 'main.py', 'fold_1_confusion_matrix.png', 'fold_2_roc_curve.png', 'tests', 'src', '.git']
